In [1]:
!pip install tensorflow-model-optimization
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 8.3 MB/s eta 0:00:00


In [3]:
# Load the TF Flowers dataset
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
num_classes = info.features['label'].num_classes

def preprocess(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0  # Normalize
    return image, label

# Prepare training and validation datasets
train_data = dataset['train'].map(preprocess).batch(32).shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_data = dataset['train'].map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [4]:
# Define a simple CNN model
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the baseline model
model.fit(train_data, validation_data=val_data, epochs=5)

9406464/9406464 [==============================] - 2s 0us/step
Epoch 1/5
115/115 [==============================] - 34s 156ms/step - loss: 0.6904 - accuracy: 0.7469 - val_loss: 0.3985 - val_accuracy: 0.8662
Epoch 2/5
115/115 [==============================] - 11s 83ms/step - loss: 0.3683 - accuracy: 0.8738 - val_loss: 0.3157 - val_accuracy: 0.8907
Epoch 3/5
115/115 [==============================] - 11s 83ms/step - loss: 0.3023 - accuracy: 0.9027 - val_loss: 0.2588 - val_accuracy: 0.9191
Epoch 4/5
115/115 [==============================] - 10s 74ms/step - loss: 0.2533 - accuracy: 0.9196 - val_loss: 0.2171 - val_accuracy: 0.9368
Epoch 5/5
115/115 [==============================] - 10s 73ms/step - loss: 0.2182 - accuracy: 0.9332 - val_loss: 0.1916 - val_accuracy: 0.9471


In [6]:
# Apply Variational-based pruning
def apply_variational_pruning(model):
    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

    # Define variational pruning parameters
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.2,
                                                                  final_sparsity=0.5,
                                                                  begin_step=0,
                                                                  end_step=1000)
    }

    pruned_model = prune_low_magnitude(model, **pruning_params)
    return pruned_model

# Apply pruning and recompile
pruned_model = apply_variational_pruning(model)
pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Add the UpdatePruningStep callback
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]

# Fine-tune pruned model with the callback
pruned_model.fit(train_data, validation_data=val_data, epochs=5, callbacks=callbacks) # Added callbacks here

Epoch 1/5
115/115 [==============================] - 63s 184ms/step - loss: 0.2634 - accuracy: 0.9098 - val_loss: 0.5600 - val_accuracy: 0.7948
Epoch 2/5
115/115 [==============================] - 16s 120ms/step - loss: 0.4753 - accuracy: 0.8354 - val_loss: 0.5384 - val_accuracy: 0.8060
Epoch 3/5
115/115 [==============================] - 15s 118ms/step - loss: 0.5728 - accuracy: 0.7970 - val_loss: 0.5490 - val_accuracy: 0.8060
Epoch 4/5
115/115 [==============================] - 16s 128ms/step - loss: 0.6858 - accuracy: 0.7526 - val_loss: 0.6589 - val_accuracy: 0.7649
Epoch 5/5
115/115 [==============================] - 15s 115ms/step - loss: 0.7578 - accuracy: 0.7098 - val_loss: 0.7268 - val_accuracy: 0.7300


In [7]:
# Evaluate pruned model
_, pruned_acc = pruned_model.evaluate(val_data)
print(f"Pruned Model Accuracy: {pruned_acc:.4f}")

115/115 [==============================] - 4s 38ms/step - loss: 0.7268 - accuracy: 0.7300
Pruned Model Accuracy: 0.7300
